In [1]:
import pandas as pd
import numpy as np

first read in and data clean, joinable on metadata so get body site

In [2]:
pca_df = pd.read_csv('sample3000_pcoa_results.csv')
print(pca_df.shape)

(2863, 2864)


In [3]:
pca_df.head()

,index,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,...,PC2854,PC2855,PC2856,PC2857,PC2858,PC2859,PC2860,PC2861,PC2862,PC2863
0,10317.000067883.26479,-0.300431,-0.089162,-0.006487,0.046056,0.015653,0.020709,-0.034481,0.020000,-0.016833,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,10317.000011941.2911,-0.377379,-0.017649,-0.028726,-0.025529,-0.119759,0.044928,-0.029181,0.024563,0.052632,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,10317.000032619.4016,-0.241194,-0.054584,-0.045798,0.122745,0.109676,0.045970,0.091546,0.067077,0.073541,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,10317.000002787.2919,-0.341264,-0.065164,0.049948,0.066130,-0.010392,-0.110999,-0.006665,-0.025091,0.024109,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,10317.000058599.25862,-0.301319,-0.095065,0.090588,0.090163,0.007951,-0.123560,0.018356,0.062150,0.116950,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [4]:
pca_df = pca_df[['index', 'PC1', 'PC2', 'PC3']]

In [5]:
pca_df.head(2)

,index,PC1,PC2,PC3
0,10317.000067883.26479,-0.300431,-0.089162,-0.006487
1,10317.000011941.2911,-0.377379,-0.017649,-0.028726


In [6]:
pca_df.columns = ['sample_name', 'PC1', 'PC2', 'PC3']

In [7]:
def meta_df_id_clean(meta_df):
    meta_df['sample_id'] = meta_df['sample_name'].apply(lambda x: x.split('.')[1])
    meta_df['sample_id'] = pd.to_numeric(meta_df['sample_id'], errors='coerce', downcast='integer')
    meta_df_clean = meta_df.dropna(subset=['sample_id']).reset_index()
    meta_df_clean['sample_id'] = meta_df_clean['sample_id'].apply(lambda x: int(x))
    return meta_df_clean

In [8]:
def biom_id_clean(biom_df):
    biom_df['sample_id'] = biom_df['sample_name'].apply(lambda x: x.split('.')[1])
    biom_df['sample_id'] = pd.to_numeric(biom_df['sample_id'], errors='coerce', downcast='integer')
    biom_join_df = biom_df.dropna(subset=['sample_id'])
    biom_join_df['sample_id'] = biom_join_df['sample_id'].apply(lambda x: int(x))
    return biom_join_df

In [9]:
pca_df = biom_id_clean(pca_df)

In [10]:
pca_df.head(2)

,sample_name,PC1,PC2,PC3,sample_id
0,10317.000067883.26479,-0.300431,-0.089162,-0.006487,67883
1,10317.000011941.2911,-0.377379,-0.017649,-0.028726,11941


In [11]:
pca_df.shape

(2863, 5)

In [12]:
meta_df = pd.read_csv('all_body_4.16.agp_only_meta.csv', low_memory=False)

In [13]:
print(meta_df.shape)

(22807, 326)


In [14]:
#meta_df.head()

In [15]:
meta_df = meta_df[['sample_id', 'env_material']]

In [16]:
PC2_df = pd.merge(pca_df, meta_df, on='sample_id')
print(PC2_df.shape)

(2863, 6)


In [17]:
PC2_df.env_material.value_counts()

sebum     1004
saliva     967
feces      892
Name: env_material, dtype: int64

In [18]:
PC2_df.head()

,sample_name,PC1,PC2,PC3,sample_id,env_material
0,10317.000067883.26479,-0.300431,-0.089162,-0.006487,67883,feces
1,10317.000011941.2911,-0.377379,-0.017649,-0.028726,11941,feces
2,10317.000032619.4016,-0.241194,-0.054584,-0.045798,32619,feces
3,10317.000002787.2919,-0.341264,-0.065164,0.049948,2787,feces
4,10317.000058599.25862,-0.301319,-0.095065,0.090588,58599,feces


In [19]:
PC2_df = PC2_df[['sample_id', 'PC1', 'PC2', 'PC3', 'env_material']]

In [20]:
PC2_df.columns = ['sample_id', 'PC1', 'PC2', 'PC3', 'label']

In [21]:
PC2_df.label.value_counts()

sebum     1004
saliva     967
feces      892
Name: label, dtype: int64

In [101]:
PC2_df.to_csv('../../data/6.2.19.pca.data/output_data/beta_pcoa_3000sample.csv')

basic logistic regression body site classification

In [35]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score

In [27]:
X = PC2_df[['PC1', 'PC2', 'PC3']]
y = PC2_df['label'].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [29]:
clf = LogisticRegression(random_state=0, solver='lbfgs', multi_class='multinomial').fit(X_train, y_train)

In [33]:
clf.score

<bound method ClassifierMixin.score of LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='multinomial',
          n_jobs=1, penalty='l2', random_state=0, solver='lbfgs',
          tol=0.0001, verbose=0, warm_start=False)>

In [34]:
train_predict = clf.predict(X_train)
test_predict = clf.predict(X_test)

In [37]:
f1_score(y_train, train_predict, average='weighted') 

0.955375615637975

In [38]:
f1_score(y_test, test_predict, average='weighted') 

0.9581281732924142